In [259]:
import re
import pandas as pd

In [260]:
filename = "example-facts-1.als"
lines = ""

In [261]:
with open(filename, "r") as f:
    lines = f.read()
print(lines)  

module alloy2cd

open util/boolean



abstract sig Vehicle   {
}

sig Car extends Vehicle {
	wheels: some Wheel,
	isdeflated: Bool
}

one sig Person {
	name: String,
	age: Int, 
	cars: some Car
}


sig Wheel  {
	cars: one Car,
	diameter: Int
}


fact wheelsFact {
	#Wheel != 2
}

fact CPFact {
	#Car > 0
	#Person >= 1
}

fact diamFact {
	all w: Wheel | gt[w.diameter, 0]
	all w: Wheel | lte[w.diameter, 7]
}

fact PersonFact {
	all p: Person | p.name in "John" + "Paula" + "Michael"
	all p: Person | p.name not in "Claudia" + "Marcos"
}


pred exp {
} run exp for 3



In [262]:
########################## PACKAGE ##########################
# find package
module = re.findall('\s*module\s+(\w+)', lines)[0]


########################## CLASSES + ATTRIBUTES ##########################

# find sigs
sigs = re.findall('(\w*)(\w*)\s*sig\s+(\w+)\s+(?:extends)?\s*(\w+)?\s*\{([^{]+)\}', lines)

classNames = [s[2] for s in sigs]
classes = []
for s in sigs:
    features = re.findall('(\w*)\s*:\s*(one|some|lone)?\s*(\w*)', s[4])
    c = {
        'name': s[2], 
        'isAbstract': 'abstract' in s[0:2],
        'extends': s[3],
        'features': [],
        'associations': []
     }
    for f in features:
        if(f[2] in classNames):
            c['associations'].append({
            'name': f[0],
            'arity': f[1],
            'target': f[2] 
            })
        else:
            c['features'].append({
                'name': f[0],
                'arity': f[1],
                'type': f[2] 
            })
    classes.append(c)
    
# cd output: classes + attributes
classesStr = ''

for c in classes:
    line = '\t{abstract}class {className}{extends}'.format(abstract='abstract ' if c['isAbstract'] else '', 
                                                           className=c['name'], 
                                                           extends = '' if (c['extends']=='') else (' extends '+c['extends']))
#     if not(c in fields.keys()):
#         classesStr+=(line+';\n')
#         continue
    fieldsTxt = ''
    for f in c['features']:
        fieldsTxt+= f' {f["type"]} {f["name"]}; '
        
    line+= f' {{{fieldsTxt}}}'

    classesStr+= (line+'\n')
    
########################## ASSOCIATIONS ##########################
associations = []

for c in classes:
    for a in c['associations']:
        a['source'] = c['name']
        a['arity2'] = '-'
        b = False
        for rel in associations:
            if rel['source']==a['target'] and a['source']==rel['target']:
                rel['type'] = '--'
                rel['arity2'] = a['arity']
                b = True
                break
        if not b:
            a['type'] = '->'
            associations.append(a)

            

def get_arity(argument):
    switcher = {
        '': '[0..*]',
        'some': '[1..*]',
        'one': '[1..1]',
        'lone': '[0..1]'
    }
    return switcher.get(argument, '')

associationsStr = ''
for a in associations:
#     association WorkPlace [1] Employee (of) -> (worksIn) Address [1..3];
    name = a['name']
    arity = get_arity(a['arity'])
    arity2 = get_arity(a['arity2'])
    src = a['source']
    tgt = a['target']
    atype = a['type']
    associationsStr += f'\tassociation {name} {src} {arity2} {atype} {tgt} {arity};\n'
    


    
    
########################## FACTS/CONSTRAINTS ##########################
# find facts
factstxt = re.findall('\s*fact\s+(\w+)\s+\{([^{]+)\}', lines)

# parse facts
facts = [{'name': f[0], 'content': f[1]} for f in factstxt]

for f in facts:
    fcontent = f['content']
    # get lines
    fcontent = fcontent.split('\n')

    # remove extra white spaces
    fcontent = [' '.join(c.split()) for c in fcontent]

    # delete empty lines
    fcontent = [c for c in fcontent if c]
    
    f['content'] = fcontent
    
def fact2ocl(fact):
    # Size fact - exp: #class1 >= 2
    fname = fact['name']
    ocl = []
    for i, c in enumerate(fact['content']):
        name = fname + str(i)
        if re.match(r"\s*#\s*(\w+)\s*(>|<|=|<=|>=|!=)\s*(\d+)", c):
            ocl.append(sizeFact2ocl(name, c))
        if re.match(r"\s*all\s*(\w+)\s*:\s*(\w+)\s*\|\s*(gt|gte|lt|lte|eq)\[\s*\1.(\w+)\s*,\s*(\d+)\s*\]", c):
            ocl.append(intCompare2ocl(name, c))
        if re.match(r"\s*all\s*(\w+)\s*:\s*(\w+)\s*\|\s*\1.(\w+)\s*(in|not\s*in)\s*\(?\s*(\S+(?:\s*\+\s*.+)*)\s*\)?", c):
            ocl.append(subset2ocl(name, c))
    return ocl

def sizeFact2ocl(name, content):
    # Size fact - exp: #class1 >= 2
    elems = re.findall(r"\s*#\s*(\w+)\s*(>|<|=|<=|>=|!=)\s*(\d+)", content)[0]
    left, op, right = elems[0], elems[1], elems[2]    
    return f'inv {name} :  self.count({left}) {op} {right}'

def intCompare2ocl(name, content):
    # int compare fact - exp: all c: class1 | gt[c.attr, 0]
    elems = re.findall(r"\s*all\s*(\w+)\s*:\s*(\w+)\s*\|\s*(gt|gte|lt|lte|eq)\[\s*\1.(\w+)\s*,\s*(\d+)\s*\]", content)[0]
    char, sig, op, attr, val = elems
    return f'context {sig} inv {name} :  {attr} {getOperator(op)} {val}'

def subset2ocl(name, content):
    # int compare fact - exp: all c: class1 | gt[c.attr, 0]
    elems = re.findall(r"\s*all\s*(\w+)\s*:\s*(\w+)\s*\|\s*\1.(\w+)\s*(in|not\s*in)\s*\(?\s*(\S+(?:\s*\+\s*.+)*)\s*\)?", content)[0]
    char, sig, attr, key, subset = elems
    subset = ','.join(subset.split('+'))
    inclusion = 'includes' if key=='in' else 'excludes'
    return f'context {sig} inv {name} :  Set {{{subset}}}->{inclusion}({attr})'

def getOperator(op):
    switcher = {
        'gt': '>',
        'gte': '>=',
        'lt': '<',
        'lte': '<=',
        'eq': '==',
        
    }
    return switcher.get(op, 'NONE')
    

for f in facts:
    f['ocl'] = fact2ocl(f)
    

factsStr = '\n'.join(['\t'+o for f in facts for o in f['ocl']])

    
# output
output = f"""
package alloy2cd;

classdiagram {module} {{

{classesStr}

{associationsStr}

{factsStr}
  
}}

"""

print(output)


package alloy2cd;

classdiagram alloy2cd {

	abstract class Vehicle {}
	class Car extends Vehicle { Bool isdeflated; }
	class Person { String name;  Int age; }
	class Wheel { Int diameter; }


	association wheels Car [1..1] -- Wheel [1..*];
	association cars Person  -> Car [1..*];


	inv wheelsFact0 :  self.count(Wheel) != 2
	inv CPFact0 :  self.count(Car) > 0
	inv CPFact1 :  self.count(Person) >= 1
	context Wheel inv diamFact0 :  diameter > 0
	context Wheel inv diamFact1 :  diameter <= 7
	context Person inv PersonFact0 :  Set {"John" , "Paula" , "Michael"}->includes(name)
	context Person inv PersonFact1 :  Set {"Claudia" , "Marcos"}->excludes(name)
  
}


